In [76]:
import glob
import cv2
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math

In [4]:
#Nothing to see here

In [58]:
class generate:
    def batch_1(img):
        """ Requires uint8 image dtype"""
        images_arrays = []
        images_arrays.append(generate.rotation(img, -90))
        images_arrays.append(generate.rotation(img, 90))
        images_arrays.append( generate.vertical_flip(img))
        images_arrays.append(generate.high_brightness(img))
        images_arrays.append(generate.low_brightness(img))
        images_arrays.append( generate.sharpen(img))

        return images_arrays

    
    def rotation(image,angle):
        height, width = image.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, scale=1)
        rotated_image = cv2.warpAffine(image,rotation_matrix,(width,height))
        return rotated_image

 
    def vertical_flip(image):
        return cv2.flip(image,0)
    
    def high_brightness(image):
        return cv2.add(image, np.ones(image.shape , dtype=np.uint8) * 40)

    def low_brightness(image):
        return cv2.subtract(image, np.ones(image.shape , dtype=np.uint8) * 40)
        

    def sharpen(image):
        kernel = np.array([ [-1,-1,-1],
                            [-1,10,-1],
                            [-1,-1,-1] ])
        return cv2.filter2D(image,-1,kernel)

In [96]:
class compile:
    path = r'src/'
    def __init__(self, classes='binary', resize_shape=(256, 256), normalise=True, load_equal_ratio=True, augmentation_stage=1):
        self.fetch_imgs_paths()
        if classes == 'binary':
            self.data_x, self.data_y = self.load_binary_classes(resize_shape=resize_shape, normalise=normalise)
        elif classes == 'all':
            self.data_x, self.data_y = self.load_by_classes(resize_shape=resize_shape)
        else :
            return ValueError('Invalid class specification')
        
            
       
    @classmethod
    def fetch_imgs_paths(self):
        self.imgs_paths = {}
        for class_folder in os.listdir(f'{self.path}'):
            self.imgs_paths[f'{class_folder}'] = os.listdir(f'{self.path}{class_folder}')
    
    @staticmethod
    def gen_pool(data_len,ratio=[0.7,.2,.1]):
        pool = []
        for index, partition in enumerate(ratio):
            pool = pool + ([index] * math.floor( data_len * partition))
            random.shuffle(pool)   
        return pool
    
    @classmethod
    def load_binary_classes(self, resize_shape, normalise, fetch_max_equal_ratio=True):
        x_dataset, y_dataset = [], []
        healthy_class_len = len(self.imgs_paths['healthy'])
        class_sample_size = int(healthy_class_len / 3)
        if fetch_max_equal_ratio:
            for class_folder in self.imgs_paths:
                if class_folder == 'healthy':
                    for img_path in self.imgs_paths[class_folder]:
                        position = random.randint(0, len(x_dataset))
                        img_array = cv2.resize(cv2.imread(self.path + class_folder + '/' + img_path), resize_shape, interpolation = cv2.INTER_AREA)
                        x_dataset.insert(position, img_array / 256.0 if normalise == True else img_array)
                        y_dataset.insert(position, 0)
                        for img in generate.batch_1(img_array):
                            new_pos = random.randint(0, len(x_dataset))
                            x_dataset.insert(new_pos, img / 256.0 if normalise == True else img_array)
                            y_dataset.insert(new_pos, 0)
                            
                        
                else:
                    sample_pool = random.sample(self.imgs_paths[class_folder], class_sample_size)
                    for img in sample_pool:
                        position = random.randint(0, len(x_dataset))
                        img_array = cv2.resize(cv2.imread(self.path + class_folder + '/' + img), resize_shape, interpolation=cv2.INTER_AREA)
                        x_dataset.insert(position, img_array / 256.0 if normalise == True else img_array)
                        y_dataset.insert(position, 1)
                        for img in generate.batch_1(img_array):
                            new_pos = random.randint(0, len(x_dataset))
                            x_dataset.insert(new_pos, img / 256.0 if normalise == True else img_array)
                            y_dataset.insert(new_pos, 1)
                        
                        
        elif fetch_max_equal_ratio == False:
            for class_folder in self.imgs_paths:
                if class_folder == 'healthy':
                    for img_path in self.imgs_paths[class_folder]:
                        position = random.randint(0, len(x_dataset))
                        img_array = cv2.resize(cv2.imread(self.path + class_folder + '/' + img_path), resize_shape, interpolation = cv2.INTER_AREA)
                        img_array = img_array / 256 if normalise == True else img_array
                        x_dataset.insert(position, img_array)
                        y_dataset.insert(position, 0)
                else:
                    for img_path in self.imgs_paths[class_folder]:
                        position = random.randint(0, len(x_dataset))
                        img_array = cv2.resize(cv2.imread(self.path + class_folder + '/' + img_path), resize_shape, interpolation = cv2.INTER_AREA)
                        img_array = img_array / 256 if normalise == True else img_array
                        x_dataset.insert(position, img_array)
                        y_dataset.insert(position, 1)
                                                  
        return x_dataset, y_dataset
    
    @classmethod
    def load_all_classes(self, resize_shape, fetch_max_equal_ratio=True, normalise=256):
        pass
    @classmethod
    def generator(self, data_x, data_y):
        for index, img in enumerate(self.data_x):
            for generic_img in generate.batch_1(img):
                position = random.randint(0, len(self.data_x)) 
                data_x.insert(position,generic_img)
                data_y.insert(position, self.data_y[index]) 
        return data_x, data_y
    @property
    def load(self, train_size=.7, test_size=.3):
        if (train_size + test_size) == 1:
            train_slice = int(len(self.x_dataset)/.7)
            return self.x_dataset[:train_slice], self.y_dataset[:train_slice] , self.x_dataset[train_slice:], self.y_dataset[train_slice:]
        elif (train_size + test_size) != 1:
            return ValueError('Invalid Split Value') 
        
    

In [59]:
img = cv2.resize(cv2.imread('src/healthy/Corn_Health (1).jpg'), (256,256), interpolation=cv2.INTER_AREA)

In [60]:
b =  generate.batch_1(img)

In [63]:
b = [1,2,3,5,6,9]

In [65]:
b.pop(2)

3

In [ ]:
def partition(dataset, part=[7,2,1]):
    pool =  'n'
    
    

In [234]:
b = {1 : [456,454], 4:[467,465]}

In [255]:
for key, items in b.items():
    print(key, items)

1 456
4 467


In [274]:
def fetch_imgs_path(path, class_encoding, equal_ratio_to_healthy, shuffle):
    class_folders = os.listdir(path)
    imgs_path_dict = {}
    for class_folder in class_folders:
        files = [path + class_folder + '/' + file for file in os.listdir(path + class_folder + '/')]
        if shuffle == True:
            random.shuffle(files)
        imgs_path_dict[str(class_folder)] = files
    if class_encoding == 'binary':
        temp_dict = {'unhealthy':[], 'healthy':[]}
        if equal_ratio_to_healthy == True:
            ratio = int(len(imgs_path_dict['healthy']) / (len(class_folders) - 1))
            for folder, img_files  in imgs_path_dict.items():
                if folder == 'healthy':
                    temp_dict['healthy'] = random.sample(imgs_path_dict[folder], len(imgs_path_dict['healthy']))
                else:
                    temp_dict['unhealthy'] = temp_dict['unhealthy'] + random.sample(imgs_path_dict[folder], ratio)
        
            imgs_path_dict = temp_dict   
            
        else:
            for class_folder in class_folder:
                if class_folder == 'healthy':
                    temp_dict['healthy'] = imgs_path_dict[class_folder]
                else:
                    temp_dict['unhealthy'] = imgs_path_dict[class_folder]
            
            imgs_path_dict = temp_dict
            
        
    return imgs_path_dict

In [273]:
curl = fetch_imgs_path(path='src/',class_encoding='binary', equal_ratio_to_healthy=True, shuffle=True)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'list'

In [268]:
for key, items in curl.items():
    print(key, len(items))

unhealthy 387
healthy 1162


In [248]:
del fetch_imgs_path